### `tf.radnom.set_seed`
- Sets the `global` random seed.

In [1]:
import tensorflow as tf

Operations that rely on a random seed actually derive it from two seeds: the `global` and `operation-level` seeds. This sets the global seed.

Its interactions with operation-level seeds is as follows:

1. If neither the `global seed` `nor` `the operation seed` is set: A randomly picked seed is used for this op.
1. If the `graph-level seed is set`, but `the operation seed is not`: The system deterministically picks an operation seed in conjunction with the graph-level seed so that it gets a unique random sequence. Within the same version of tensorflow and user code, this sequence is deterministic. However across different versions, this sequence might change. **If the code depends on particular seeds to work, specify both graph-level and operation-level seeds explicitly.**
1. If `the operation seed is set`, but `the global seed is not` set: A default global seed and the specified operation seed are used to determine the random sequence.
1. If `both the global and the operation seed are set`: Both seeds are used in conjunction to determine the random sequence.

#### 1st case

If neither `the global seed nor the operation seed is set`, we get different results for `every call` to the random op and `every re-run` of the program:

In [2]:
print(tf.random.uniform([1]))
print(tf.random.uniform([1]))

tf.Tensor([0.5460048], shape=(1,), dtype=float32)
tf.Tensor([0.8689755], shape=(1,), dtype=float32)


In [ ]:
# 1st
# tf.Tensor([0.5457765], shape=(1,), dtype=float32)   # generate 'A1'
# tf.Tensor([0.15078557], shape=(1,), dtype=float32)  # generate 'A2'  

# 2nd - now close the program and run it again
# tf.Tensor([0.5460048], shape=(1,), dtype=float32)   # generate 'A3'
# tf.Tensor([0.8689755], shape=(1,), dtype=float32)   # generate 'A4'

#### 2nd case

If `the global seed is set` but `the operation seed is not` set, we get different results for `every call` to the random op, but the `same sequence for every re-run` of the program.

In [2]:
tf.random.set_seed(1234)
print(tf.random.uniform([1]))
print(tf.random.uniform([1]))

tf.Tensor([0.5380393], shape=(1,), dtype=float32)
tf.Tensor([0.3253647], shape=(1,), dtype=float32)


In [ ]:
# 1st
# tf.Tensor([0.5380393], shape=(1,), dtype=float32)   # generates 'A1'
# tf.Tensor([0.3253647], shape=(1,), dtype=float32)   # generates 'A2'

# 2nd - now close the program and run it again
# tf.Tensor([0.5380393], shape=(1,), dtype=float32)   # generates 'A1'
# tf.Tensor([0.3253647], shape=(1,), dtype=float32)   # generates 'A2'

The reason we get 'A2' instead 'A1' on the second call of `tf.random.uniform` is because the second call uses a `different operation seed`.  
Note that `tf.function` acts like a re-run of a program in this case. When the global seed is set but operation seeds are not set, the sequence of random numbers are the same for each `tf.function`. For example:

In [3]:
tf.random.set_seed(1234)

@tf.function
def f():
    a = tf.random.uniform([1])
    b = tf.random.uniform([1])
    return a, b

@tf.function
def g():
    a = tf.random.uniform([1])
    b = tf.random.uniform([1])
    return a, b

print(f())
print(g())

(<tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.13047123], dtype=float32)>, <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.1689806], dtype=float32)>)
(<tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.13047123], dtype=float32)>, <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.1689806], dtype=float32)>)


In [ ]:
# (numpy=array([0.13047123], numpy=array([0.1689806])   # prints '(A1, A2)'
# (numpy=array([0.13047123], numpy=array([0.1689806])   # prints '(A1, A2)'

#### 3rd case

If `the operation is set`, we get different results for `every call` to the random op, but the `same sequence for every re-run` of the program:

In [2]:
print(tf.random.uniform([1], seed=1))
print(tf.random.uniform([1], seed=1))

tf.Tensor([0.2390374], shape=(1,), dtype=float32)
tf.Tensor([0.22267115], shape=(1,), dtype=float32)


In [ ]:
# 1st
# tf.Tensor([0.2390374], shape=(1,), dtype=float32)    # generates 'A1'
# tf.Tensor([0.22267115], shape=(1,), dtype=float32)   # generates 'A2'

# 2nd - now close the program and run it again
# tf.Tensor([0.2390374], shape=(1,), dtype=float32)    # generates 'A1'
# tf.Tensor([0.22267115], shape=(1,), dtype=float32)   # generates 'A2'

The reason we get 'A2' instead 'A1' on the second call of `tf.random.uniform` is because the same `tf.random.uniform` kernel (i.e. internal representation) is used by TensorFlow for all calls of it with the same arguments, and the kernel maintains an internal counter which is incremented every time it is executed, generating different results.

Calling `tf.random.set_seed` will reset any such counters:

In [45]:
tf.random.set_seed(1234)
print(tf.random.uniform([1], seed=1))  # generates 'A1'
print(tf.random.uniform([1], seed=1))  # generates 'A2'
tf.random.set_seed(1234)
print(tf.random.uniform([1], seed=1))  # generates 'A1'
print(tf.random.uniform([1], seed=1))  # generates 'A2'

tf.Tensor([0.1689806], shape=(1,), dtype=float32)
tf.Tensor([0.7539084], shape=(1,), dtype=float32)
tf.Tensor([0.1689806], shape=(1,), dtype=float32)
tf.Tensor([0.7539084], shape=(1,), dtype=float32)


When multiple identical random ops are wrapped in a `tf.function`, their behaviors change because the ops no long share the same counter. For example:

In [71]:
@tf.function
def foo():
    a = tf.random.uniform([1], seed=1)
    b = tf.random.uniform([1], seed=1)
    return a, b
print(foo())
print(foo())

@tf.function
def bar():
    a = tf.random.uniform([1])
    b = tf.random.uniform([1])
    return a, b
print(bar())
print(bar())

(<tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.1689806], dtype=float32)>, <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.1689806], dtype=float32)>)
(<tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.7539084], dtype=float32)>, <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.7539084], dtype=float32)>)
(<tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.13047123], dtype=float32)>, <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.1689806], dtype=float32)>)
(<tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.6087816], dtype=float32)>, <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.7539084], dtype=float32)>)


In [ ]:
# (numpy=array([0.1689806], numpy=array([0.1689806])   # prints '(A1, A1)'
# (numpy=array([0.7539084],  numpy=array([0.7539084])  # prints '(A2, A2)'
# (numpy=array([0.13047123], numpy=array([0.1689806])  # prints '(A1, A2)'
# (<numpy=array([0.6087816], numpy=array([0.7539084])  # prints '(A3, A4)'

In [77]:
def foo():
    tf.random.set_seed(1234)
    a = tf.random.uniform([1], seed=1)
    tf.random.set_seed(1234)
    b = tf.random.uniform([1], seed=1)
    return a, b
print(foo())
print(foo())

def bar():
    tf.random.set_seed(1234)
    a = tf.random.uniform([1])
    tf.random.set_seed(1234)
    b = tf.random.uniform([1])
    return a, b
print(bar())
print(bar())

(<tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.1689806], dtype=float32)>, <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.1689806], dtype=float32)>)
(<tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.1689806], dtype=float32)>, <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.1689806], dtype=float32)>)
(<tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.5380393], dtype=float32)>, <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.5380393], dtype=float32)>)
(<tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.5380393], dtype=float32)>, <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.5380393], dtype=float32)>)


In [80]:
tf.random.set_seed(1234)
print(tf.random.uniform([1]))
tf.random.set_seed(1234)
print(tf.random.uniform([1]))

tf.Tensor([0.5380393], shape=(1,), dtype=float32)
tf.Tensor([0.5380393], shape=(1,), dtype=float32)
